# Title
\[insert intro + motivation\]

In [ ]:
import sqlite3
import pandas as pd
import folium

## Importing Data


In [2]:
# tables are [('station',), ('status',), ('trip',), ('weather',)]
con = sqlite3.connect("database.sqlite")

# get tables
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor = con.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('station',), ('status',), ('trip',), ('weather',)]


In [3]:
station = pd.read_sql_query("SELECT * from station", con)
status = pd.read_sql_query("SELECT * from status", con)
trip = pd.read_sql_query("SELECT * from trip", con)
weather = pd.read_sql_query("SELECT * from weather", con)

In [4]:
# Columns in each table
station # id, name, lat, long, dock_count, city, installation_date
status # station_id, bikes_available, docks_available, time
trip 
"""(['id', 'duration', 'start_date', 'start_station_name',
       'start_station_id', 'end_date', 'end_station_name', 'end_station_id',
       'bike_id', 'subscription_type', 'zip_code'],
      dtype='object')"""
weather 
""" (['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f',
       'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f',
       'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'max_gust_speed_mph', 'precipitation_inches',
       'cloud_cover', 'events', 'wind_dir_degrees', 'zip_code'],
      dtype='object')"""

status


NameError: name 'status' is not defined

## Data Cleaning and Manipulation (reword me)

In [ ]:
# mark all the stations
map_osm = folium.Map(location=[37.60, -122.29], zoom_start=10)

for index, station_info in station.iterrows():
    tooltip_text = station_info['name'] + "\n"
    folium.Marker(location=[station_info["lat"], station_info["long"]], tooltip=tooltip_text).add_to(map_osm)
    
map_osm

In [ ]:
# Convert time from object to datetime and remove seconds
status["time"] = pd.to_datetime(status["time"]).dt.floor('Min')
status.head()

## Data Exploration

## Data Analysis and Hypothesis Testing

## Predicting [something] with Machine Learning

## Conclusion